In [49]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [50]:
def load_img(filename):
    return cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)

In [51]:
paintings = [load_img(f"../PGMData/imgs/schilderij_{i}.jpg") for i in range(0, 2269)]
photos = [load_img(f"../PGMData/imgs/foto_{i}.jpg") for i in range(0, 2269)]
drawings = [load_img(f"../PGMData/imgs/tekening_{i}.jpg") for i in range(0, 2269)]
prints = [load_img(f"../PGMData/imgs/prent_{i}.jpg") for i in range(0, 2269)]
print(len(paintings), len(photos), len(drawings), len(prints))
print(paintings[0].shape)
print(photos[0].shape)
print(drawings[0].shape)
print(prints[0].shape)

2269 2269 2269 2269
(500, 364, 3)
(248, 500, 3)
(337, 500, 3)
(362, 500, 3)


In [152]:
# For constant # of pixels
SIZE = 128
data_paintings = [cv2.resize(img, (SIZE, SIZE), interpolation=cv2.INTER_AREA).reshape(-1) for img in paintings]
data_photos = [cv2.resize(img, (SIZE, SIZE), interpolation=cv2.INTER_AREA).reshape(-1) for img in photos]
data_drawings = [cv2.resize(img, (SIZE, SIZE), interpolation=cv2.INTER_AREA).reshape(-1) for img in drawings]
data_prints = [cv2.resize(img, (SIZE, SIZE), interpolation=cv2.INTER_AREA).reshape(-1) for img in prints]

In [153]:
print(data_paintings[0].shape)
print(data_photos[0].shape)
print(data_drawings[0].shape)
print(data_prints[0].shape)

(49152,)
(49152,)
(49152,)
(49152,)


In [154]:
train = np.stack(tuple(data_paintings + data_photos + data_drawings + data_prints), axis=0)
test = [0 for _ in range(len(data_paintings))] + [1 for _ in range(len(data_photos))] + [2 for _ in range(len(data_drawings))] + [3 for _ in range(len(data_prints))]
X_train, X_test, y_train, y_test = train_test_split(train, test, test_size=0.1, random_state=42)
print(train.shape, len(test))
print(X_train.shape, len(y_train))
print(X_test.shape, len(y_test))

(9076, 49152) 9076
(8168, 49152) 8168
(908, 49152) 908


In [155]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

In [156]:
y_train

tensor([0, 1, 1,  ..., 2, 0, 3])

In [163]:
class LR(nn.Module):
    def __init__(self, input_features, n_classes):
        super(LR, self).__init__()
        self.linear = nn.Linear(input_features, n_classes)
        
    def forward(self, x):
        return self.linear(x)
    
    def get_probs(self, x):
        out = self.linear(x)
        out = F.softmax(out, dim=1)
        return out

In [164]:
model = LR(X_train.shape[1], 4)
print(X_train)

tensor([[ 71.,  65.,  52.,  ...,  75.,  69.,  61.],
        [170., 153., 107.,  ..., 167., 152., 109.],
        [236., 233., 214.,  ..., 237., 236., 218.],
        ...,
        [236., 230., 215.,  ..., 235., 229., 211.],
        [ 58.,  55.,  48.,  ...,  32.,  31.,  29.],
        [247., 244., 225.,  ..., 246., 243., 224.]])


In [165]:
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters())
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(X_train)

    # Compute and print loss
    loss = criterion(y_pred, y_train)
    if t % 50 == 0:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 1063232.625
50 1714135.875
100 2356326.75
150 6723216.0
200 2229403.75
250 745520.4375
300 4289069.5
350 1082893.875
400 1806064.625
450 2000958.625


In [166]:
y_pred = model.get_probs(X_test)

In [167]:
correct = 0
total = 0
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    total += y_test.size(0)
    correct += (predicted == y_test).sum().item()

print(f'Accuracy of the network on the {y_test.size(0)} test images: {100 * correct / total} %')

Accuracy of the network on the 908 test images: 39.647577092511014 %


In [168]:
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    c = (predicted == y_test).squeeze()
    for i in range(X_test.size(0)):
        label = y_test[i]
        class_correct[label] += c[i].item()
        class_total[label] += 1
        

classes = ["Paintings", "Photos", "Drawings", "Prints"]
for i in range(4):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of Paintings :  0 %
Accuracy of Photos : 79 %
Accuracy of Drawings : 82 %
Accuracy of Prints :  0 %
